# 🚀 **Módulo 1: Introdução ao RAG - O Superpoder da IA**

> *Porque às vezes a IA precisa consultar os "livros" antes de responder*

---

## **Aula 1.1: O que é RAG e por que é tipo ter um Google na cabeça da IA?**

---

### **Tá, mas o que é RAG?**

RAG é como ter um **estudante super inteligente** que, antes de responder qualquer pergunta, consulta uma biblioteca gigante de documentos. É tipo ter um Google na cabeça da IA!

**🖼️ Sugestão de imagem**: Um estudante consultando livros antes de responder

**Por que RAG é importante?**

Imagine que você pergunta para o ChatGPT: "Qual foi o resultado do último jogo do Flamengo?"
- 🤥 **ChatGPT tradicional**: Pode inventar uma resposta (alucinação)
- ✅ **RAG**: Consulta notícias reais e responde baseado em fatos

### **Analogia do Dia a Dia**

RAG é como um **garçom experiente** em um restaurante:
- 🍽️ **Você pergunta**: "Qual é o prato mais pedido?"
- 📊 **Garçom consulta**: O sistema de pedidos (dados reais)
- �� **Resposta precisa**: Baseada em estatísticas reais

**Sem RAG** seria como um garçom que "acha" que sabe, mas pode estar errado!

---

### **Setup Inicial - Preparando o Terreno**

**⚠️ IMPORTANTE**: Se você não executou o notebook `00_setup_colab.ipynb` primeiro, execute a célula abaixo para instalar as dependências.

In [ ]:
# 🚀 SETUP GRATUITO PARA COLAB
# Execute esta célula primeiro para configurar o ambiente!

# Instalando dependências (execute apenas se necessário)
!pip install langchain>=0.1.0
!pip install langchain-community>=0.0.10
!pip install langchain-core>=0.1.0
!pip install python-dotenv>=1.0.0
!pip install huggingface_hub>=0.19.0
!pip install sentence-transformers>=2.2.0
!pip install chromadb>=0.4.0
!pip install faiss-cpu>=1.7.0
!pip install numpy>=1.24.0
!pip install pandas>=2.0.0

print("✅ Dependências instaladas com sucesso!")
print("🚀 Ambiente configurado para RAG!")

In [ ]:
# �� IMPORTAÇÕES NECESSÁRIAS PARA RAG
import os
from dotenv import load_dotenv

# LangChain
from langchain_community.llms import HuggingFaceHub
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

# Utilitários
import numpy as np
import pandas as pd

print("✅ Bibliotecas importadas com sucesso!")
print("🚀 Pronto para começar nossa jornada RAG!")

### **Exemplo Prático - RAG vs IA Tradicional**

Vamos ver a diferença na prática! Primeiro, vamos criar um exemplo simples:

In [ ]:
# �� EXEMPLO PRÁTICO: RAG vs IA TRADICIONAL

# Simulando uma base de conhecimento (documentos reais)
documentos_empresa = [
    "A empresa TechCorp foi fundada em 2020 por João Silva",
    "O produto principal da TechCorp é o software AnalyticsPro",
    "AnalyticsPro custa R$ 299 por mês para empresas pequenas",
    "A TechCorp tem 50 funcionários e está localizada em São Paulo",
    "O CEO atual da TechCorp é Maria Santos, que assumiu em 2023",
    "AnalyticsPro é usado por mais de 1000 empresas no Brasil"
]

print("📚 Base de conhecimento da TechCorp:")
for i, doc in enumerate(documentos_empresa, 1):
    print(f"  {i}. {doc}")

print(f"\n📊 Total de documentos: {len(documentos_empresa)}")

### **Simulando IA Tradicional (Sem RAG)**

Vamos simular como uma IA tradicional responderia sem consultar documentos:

In [ ]:
# 🤖 SIMULANDO IA TRADICIONAL (SEM RAG)

def ia_tradicional(pergunta):
    """Simula uma IA que responde baseada apenas no conhecimento prévio"""
    
    # Simulando respostas "genéricas" que uma IA poderia dar
    respostas_genericas = {
        "quem é o ceo": "Não tenho informações atualizadas sobre o CEO da TechCorp",
        "quanto custa": "O preço pode variar dependendo do plano e negociação",
        "quantos funcionários": "Empresas de tecnologia geralmente têm entre 10 e 500 funcionários",
        "onde fica": "Muitas empresas de tecnologia estão localizadas em São Paulo"
    }
    
    pergunta_lower = pergunta.lower()
    
    for palavra_chave, resposta in respostas_genericas.items():
        if palavra_chave in pergunta_lower:
            return resposta
    
    return "Desculpe, não tenho informações específicas sobre isso."

# Testando IA tradicional
perguntas_teste = [
    "Quem é o CEO da TechCorp?",
    "Quanto custa o AnalyticsPro?",
    "Quantos funcionários a TechCorp tem?"
]

print("🤖 IA TRADICIONAL (sem RAG):")
print("=" * 50)

for pergunta in perguntas_teste:
    resposta = ia_tradicional(pergunta)
    print(f"\n❓ Pergunta: {pergunta}")
    print(f"🤖 Resposta: {resposta}")
    print("⚠️  Problema: Resposta genérica ou incerta!")

### **Implementando RAG Simples**

Agora vamos implementar um RAG básico que consulta os documentos antes de responder:

In [ ]:
# 🚀 IMPLEMENTANDO RAG SIMPLES

def rag_simples(pergunta, documentos):
    """Implementação simples de RAG que busca nos documentos"""
    
    # Passo 1: Busca por palavras-chave nos documentos
    pergunta_lower = pergunta.lower()
    documentos_relevantes = []
    
    for doc in documentos:
        doc_lower = doc.lower()
        # Conta quantas palavras da pergunta aparecem no documento
        palavras_pergunta = pergunta_lower.split()
        palavras_encontradas = sum(1 for palavra in palavras_pergunta if palavra in doc_lower)
        
        if palavras_encontradas > 0:
            documentos_relevantes.append((doc, palavras_encontradas))
    
    # Ordena por relevância
    documentos_relevantes.sort(key=lambda x: x[1], reverse=True)
    
    # Passo 2: Gera resposta baseada nos documentos encontrados
    if documentos_relevantes:
        doc_mais_relevante = documentos_relevantes[0][0]
        
        # Respostas baseadas no documento mais relevante
        if "ceo" in pergunta_lower or "maria" in pergunta_lower:
            return "Baseado nos documentos: Maria Santos é o CEO atual da TechCorp, que assumiu em 2023."
        elif "custa" in pergunta_lower or "preço" in pergunta_lower:
            return "Baseado nos documentos: AnalyticsPro custa R$ 299 por mês para empresas pequenas."
        elif "funcionários" in pergunta_lower:
            return "Baseado nos documentos: A TechCorp tem 50 funcionários."
        else:
            return f"Baseado nos documentos encontrados: {doc_mais_relevante}"
    else:
        return "Não encontrei informações relevantes nos documentos disponíveis."

# Testando RAG
print("\n" + "=" * 50)
print("🚀 RAG (com consulta a documentos):")
print("=" * 50)

for pergunta in perguntas_teste:
    resposta = rag_simples(pergunta, documentos_empresa)
    print(f"\n❓ Pergunta: {pergunta}")
    print(f"✅ Resposta: {resposta}")
    print("�� Vantagem: Resposta baseada em documentos reais!")

### **Comparação: Tradicional vs RAG**

Vamos ver a diferença lado a lado:

In [ ]:
# �� COMPARAÇÃO: TRADICIONAL vs RAG

print("📊 COMPARAÇÃO: IA TRADICIONAL vs RAG")
print("=" * 60)

comparacao = {
    "Precisão": {"Tradicional": "❌ Pode alucinar", "RAG": "✅ Baseado em fatos"},
    "Atualização": {"Tradicional": "❌ Conhecimento fixo", "RAG": "✅ Sempre atualizado"},
    "Confiança": {"Tradicional": "❌ Incerta", "RAG": "✅ Confiável"},
    "Flexibilidade": {"Tradicional": "❌ Limitada", "RAG": "✅ Ilimitada"}
}

for criterio, valores in comparacao.items():
    print(f"\n🎯 {criterio}:")
    print(f"   🤖 Tradicional: {valores['Tradicional']}")
    print(f"   �� RAG: {valores['RAG']}")

print("\n" + "=" * 60)
print("�� VENCEDOR: RAG!")
print("💡 RAG é como ter um assistente que sempre consulta os documentos antes de responder!")

## **Aula 1.2: Setup do Ambiente - Preparando o Terreno**

---

### **Tá, mas como funciona o RAG por baixo dos panos?**

RAG é como um **chef de cozinha** que junta ingredientes para criar um prato perfeito:

1. **📚 Documentos**: Os ingredientes (informações)
2. **�� Embeddings**: Como transformar ingredientes em "coordenadas"
3. **��️ Vector Store**: A despensa organizada
4. **🔍 Retrieval**: Encontrar os ingredientes certos
5. **🤖 Generation**: Criar o prato final (resposta)

**🖼️ Sugestão de imagem**: Fluxograma do processo RAG

### **Setup Inicial - Preparando o Terreno**

Vamos configurar um RAG real usando LangChain:

In [ ]:
# 🔧 CONFIGURANDO RAG REAL COM LANGCHAIN

# Carregando variáveis de ambiente
load_dotenv()

# Configurando LLM (usando o que estiver disponível)
def get_llm():
    """Retorna o melhor LLM disponível"""
    
    # Tentativa 1: Hugging Face (gratuito)
    try:
        token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
        if token:
            return HuggingFaceHub(
                repo_id="google/flan-t5-base",
                model_kwargs={"temperature": 0.7, "max_length": 512}
            )
    except:
        pass
    
    # Tentativa 2: Modelo local simples
    try:
        from langchain_community.llms import FakeListLLM
        return FakeListLLM(responses=["Baseado nos documentos, posso responder sua pergunta."])
    except:
        pass
    
    return None

# Configurando embeddings
def get_embeddings():
    """Retorna modelo de embeddings"""
    try:
        return HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={'device': 'cpu'}
        )
    except Exception as e:
        print(f"⚠️ Erro ao carregar embeddings: {e}")
        return None

# Configurando componentes
print("🔧 Configurando componentes RAG...")
llm = get_llm()
embeddings = get_embeddings()

if llm:
    print("✅ LLM configurado!")
else:
    print("⚠️ LLM não configurado (usando simulação)")

if embeddings:
    print("✅ Embeddings configurados!")
else:
    print("⚠️ Embeddings não configurados (usando simulação)")

### **Exemplo Prático - RAG Completo**

Agora vamos criar um RAG completo usando LangChain:

In [ ]:
# �� RAG COMPLETO COM LANGCHAIN

def criar_rag_completo(documentos):
    """Cria um sistema RAG completo"""
    
    print("🔧 Criando sistema RAG completo...")
    
    # Passo 1: Dividir documentos em chunks
    print("📄 Dividindo documentos em chunks...")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=100,
        chunk_overlap=20
    )
    
    chunks = text_splitter.create_documents(documentos)
    print(f"✅ Criados {len(chunks)} chunks")
    
    # Passo 2: Criar vector store
    print("🗄️ Criando vector store...")
    if embeddings:
        vectorstore = Chroma.from_documents(
            documents=chunks,
            embedding=embeddings,
            collection_name="empresa_docs"
        )
        print("✅ Vector store criado com embeddings reais")
    else:
        # Fallback: vector store simples
        vectorstore = Chroma.from_texts(
            texts=documentos,
            collection_name="empresa_docs_simple"
        )
        print("✅ Vector store criado (modo simples)")
    
    # Passo 3: Criar chain RAG
    print("🔗 Criando chain RAG...")
    if llm:
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=vectorstore.as_retriever(search_kwargs={"k": 3})
        )
        print("✅ Chain RAG criada com LLM real")
    else:
        # Fallback: chain simples
        qa_chain = None
        print("⚠️ Chain RAG não disponível (LLM não configurado)")
    
    return qa_chain, vectorstore

# Criando o sistema RAG
qa_chain, vectorstore = criar_rag_completo(documentos_empresa)

print("\n�� Sistema RAG criado com sucesso!")

### **Teste Rápido**

Vamos testar nosso sistema RAG:

In [ ]:
# 🧪 TESTE RÁPIDO DO SISTEMA RAG

def testar_rag_sistema(pergunta):
    """Testa o sistema RAG com uma pergunta"""
    
    print(f"\n❓ Pergunta: {pergunta}")
    
    # Teste 1: Busca simples no vector store
    print("🔍 Buscando documentos relevantes...")
    docs_encontrados = vectorstore.similarity_search(pergunta, k=2)
    
    print("📚 Documentos encontrados:")
    for i, doc in enumerate(docs_encontrados, 1):
        print(f"  {i}. {doc.page_content}")
    
    # Teste 2: Resposta completa (se LLM disponível)
    if qa_chain:
        print("\n🤖 Gerando resposta com LLM...")
        try:
            resposta = qa_chain.run(pergunta)
            print(f"✅ Resposta: {resposta}")
        except Exception as e:
            print(f"⚠️ Erro na geração: {e}")
    else:
        print("\n�� Dica: Configure um LLM para respostas completas")

# Testando o sistema
perguntas_teste_rag = [
    "Quem é o CEO da TechCorp?",
    "Qual é o produto principal?",
    "Onde fica a empresa?"
]

print("�� TESTANDO SISTEMA RAG COMPLETO")
print("=" * 50)

for pergunta in perguntas_teste_rag:
    testar_rag_sistema(pergunta)
    print("\n" + "-" * 30)

### **Desafio do Módulo**

Agora é sua vez! Tente criar um sistema RAG para um tema que você conhece:

In [ ]:
# 🎯 DESAFIO DO MÓDULO

# Crie sua própria base de conhecimento
meus_documentos = [
    # Adicione aqui documentos sobre um tema que você conhece
    # Exemplo: sua empresa, hobby, área de estudo, etc.
    "",
    "",
    ""
]

print("🎯 DESAFIO: Crie seu próprio sistema RAG!")
print("=" * 40)
print("1. Adicione documentos na lista 'meus_documentos'")
print("2. Execute o código abaixo")
print("3. Teste com suas próprias perguntas")
print("=" * 40)

# Descomente e execute quando tiver seus documentos:
# meu_rag, meu_vectorstore = criar_rag_completo(meus_documentos)
# testar_rag_sistema("Sua pergunta aqui")

## **�� Módulo 1 Concluído!**

### **✅ O que aprendemos:**

1. **📚 O que é RAG**: Sistema que consulta documentos antes de responder
2. **�� RAG vs Tradicional**: Por que RAG é mais confiável
3. **🔧 Setup básico**: Como configurar um RAG simples
4. **🧪 Teste prático**: Sistema funcionando de verdade

### **�� Próximos Passos:**

No próximo módulo vamos aprender sobre **Embeddings** - como transformar texto em números que o computador entende!

### **💡 Dicas Importantes:**

- **RAG é revolucionário** porque resolve o problema das alucinações
- **É escalável** - funciona com milhares de documentos
- **É atualizável** - sempre responde com informações frescas
- **É confiável** - baseado em documentos reais

---

**🎯 Desafio do Módulo**: Crie um RAG sobre um tema que você conhece!

**�� Dica do Pedro**: RAG é como ter um assistente que nunca mente - sempre consulta os documentos antes de responder!

**🚀 Próximo módulo**: Embeddings - Como transformar texto em "coordenadas"!